<a href="https://colab.research.google.com/github/wgova/time_series_trade/blob/oec_api/data_prep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')
!pip install oec
import oec
import pandas as pd
import numpy as np
import os, os.path, csv, requests, pathlib
import math
from datetime import datetime

#%matplotlib inline

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


## List of industries to add to the data

In [0]:
def build_call(*args):
    call_url = 'http://atlas.media.mit.edu/'
    for val in args:
        call_url += str(val) + '/'
    return call_url


def request_data(call_url):
    r = requests.get(call_url)
    response_dict = r.json()
    json_list = response_dict['data']  # list of dicts containing data
    return json_list


def get_countries(filename=None):
    call = build_call('attr', 'country')
    json_list = request_data(call)
    if filename is not None:
        data_to_csv(json_list, filename)
    return json_list


def get_products(classification, filename=None):
    call = build_call('attr', classification)
    json_list = request_data(call)
    if filename is not None:
        data_to_csv(json_list, filename)
    return json_list


def get_trade(classification, trade_flow, year, origin, destination,product, filename=None):
    call = build_call(classification, trade_flow, year, origin, destination,
                      product)
    json_list = request_data(call)
    if filename is not None:
        data_to_csv(json_list, filename)
    return json_list


def trade_params(classification, trade_flow, year, origin, destination,product):
    parameters = {'classification': classification,
                  'trade_flow': trade_flow,
                  'year': year,
                  'origin': origin,
                  'destination': destination,
                  'product': product}
    return parameters


def get_header(json_list):
    header = set()
    for dict in json_list:
        header.update(dict.keys())
    header = list(header)
    header.sort()
    return header


def create_csv(json_list, filename):
    with open(filename, 'w') as csvfile:
        header = get_header(json_list)
        cw = csv.writer(csvfile)
        cw.writerow(header)
        for dict in json_list:
            row = dict_to_list(dict, header)
            cw.writerow(row)


def dict_to_list(dict, header):
    row = []
    for field in header:
        if field in dict:
            row.append(str(dict[field]))
        else:
            row.append(None)
    return row

## Generate dataset

In [0]:
trade_codes = ['6519',
               '6531',
               '8471','6571','7849','7810','7842','7764','7723','7522','2924','5419','5417','7763','7711','7188']
PATH = '/content/drive/My Drive/phd_data/sitc'
os.chdir(PATH)
trade_classification = 'sitc'
def countries_and_products():
  countries = oec.get_countries()
  products = oec.get_products(trade_classification)
  create_csv(countries,f'{PATH}/list_countries.csv')
  create_csv(products,f'{PATH}/list_products.csv')

In [0]:
countries_and_products()

In [0]:
import os.path
from os import path
def download_data(list_of_codes):
  for i in trade_codes:
    csv_name = f"{trade_classification}-{i}"
    # Set parameters to extract top exporters from the API documentation
    params = {'classification': trade_classification,
          'trade_flow': 'export',
          'year': 'all',
          'origin': 'show', #Set origin to all for comparison
          'destination': 'all',
          'product': i} #Set code for product of interest
    print(f'Data for {i} processed.....')
    oec_data = get_trade(**params)
    # Save the results in CSV file
    if path.exists(PATH):
      create_csv(oec_data,f'{csv_name}.csv')#,locals(),globals())
    else:
      print('Choose destination directory')

In [6]:
download_data(trade_codes)

Data for 6519 processed.....
Data for 6531 processed.....
Data for 8471 processed.....
Data for 6571 processed.....
Data for 7849 processed.....
Data for 7810 processed.....
Data for 7842 processed.....
Data for 7764 processed.....
Data for 7723 processed.....
Data for 7522 processed.....
Data for 2924 processed.....
Data for 5419 processed.....
Data for 5417 processed.....
Data for 7763 processed.....
Data for 7711 processed.....
Data for 7188 processed.....


In [0]:
def create_df():
  trade_data = [i for i in os.listdir(os.getcwd()) if i.endswith('.csv') and 'sitc' in i]
  trade_history = []
  for file in trade_data:
    file_name = pd.read_csv(file)
    trade_history.append(file_name)
  products_dataframe = pd.concat(trade_history)
  return products_dataframe

In [0]:
trade_dframe = create_df()

In [24]:
trade_dframe

,export_rca,export_val,export_val_growth_pct,export_val_growth_pct_5,export_val_growth_val,export_val_growth_val_5,import_rca,import_val,import_val_growth_pct,import_val_growth_pct_5,import_val_growth_val,import_val_growth_val_5,origin_id,sitc_id,sitc_id_len,year
0,0.020916,3000.0,NaN,NaN,NaN,NaN,NaN,24000.0,NaN,NaN,NaN,NaN,afciv,606519,6.0,1962.0
1,0.025602,6000.0,NaN,NaN,NaN,NaN,NaN,21000.0,NaN,NaN,NaN,NaN,afcod,606519,6.0,1962.0
2,0.001952,1000.0,NaN,NaN,NaN,NaN,NaN,207000.0,NaN,NaN,NaN,NaN,afdza,606519,6.0,1962.0
3,0.078182,11000.0,NaN,NaN,NaN,NaN,NaN,36000.0,NaN,NaN,NaN,NaN,afegy,606519,6.0,1962.0
4,0.141766,17000.0,NaN,NaN,NaN,NaN,NaN,129000.0,NaN,NaN,NaN,NaN,afken,606519,6.0,1962.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6566,0.000203,2180.0,NaN,NaN,NaN,NaN,0.380628,3387852.0,NaN,NaN,NaN,NaN,saguy,117522,6.0,2017.0
6567,0.000840,303405.5,NaN,NaN,NaN,NaN,0.985404,298998072.5,NaN,NaN,NaN,NaN,saper,117522,6.0,2017.0
6568,0.000292,15463.0,NaN,NaN,NaN,NaN,1.595950,147727869.0,NaN,NaN,NaN,NaN,sapry,117522,6.0,2017.0
6569,0.001449,18817.0,NaN,NaN,NaN,NaN,0.250366,2111172.5,NaN,NaN,NaN,NaN,sasur,117522,6.0,2017.0
